In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## LIBRARIES

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import torch
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [ ]:
import argparse

In [ ]:
device = 'gpu' if torch.cuda.is_available() else 'cuda'
best_acc = 0  
start_epoch = 0  

## DATA LOADING AND PREPARATION

In [ ]:
print('==> Preparing data..')

# Define transformations
transform_train = transforms.Compose([
    transforms.Resize((24,24)),  # Resize to a smaller size
    transforms.RandomCrop(24, padding=4),
    transforms.ToTensor(),
])


# Load your dataset
trainset = ImageFolder(root='/content/drive/MyDrive/Python_Training_Dataset/blood_resnext/train', transform=transform_train)
trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = ImageFolder(root='/content/drive/MyDrive/Python_Training_Dataset/blood_resnext/test', transform=transform_test)
testloader = DataLoader(testset, batch_size=25, shuffle=False, num_workers=2)

# Define class names (modify based on your classes)
classes = ('Benign', 'Pro','Pre','Early')

==> Preparing data..


## RESNEXT

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Block(nn.Module):
    '''Grouped convolution block.'''
    expansion = 2

    def __init__(self, in_planes, cardinality=32, bottleneck_width=4, stride=1):
        super(Block, self).__init__()
        group_width = cardinality * bottleneck_width
        self.conv1 = nn.Conv2d(in_planes, group_width, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(group_width)
        self.conv2 = nn.Conv2d(group_width, group_width, kernel_size=3, stride=stride, padding=1, groups=cardinality, bias=False)
        self.bn2 = nn.BatchNorm2d(group_width)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*group_width:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*group_width, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*group_width)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNeXt(nn.Module):
    def __init__(self, num_blocks, cardinality, bottleneck_width, num_classes=4):
        super(ResNeXt, self).__init__()
        self.cardinality = cardinality
        self.bottleneck_width = bottleneck_width
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(num_blocks[0], 1)
        self.layer2 = self._make_layer(num_blocks[1], 2)
        self.layer3 = self._make_layer(num_blocks[2], 2)
        self.layer4 = self._make_layer(num_blocks[3], 2)
        self.linear = nn.Linear(cardinality*bottleneck_width*8, num_classes)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 6)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [ ]:
def ResNeXt29_2x64d():
    return ResNeXt(num_blocks=[3,3,3], cardinality=32, bottleneck_width=4)

In [ ]:
net = ResNeXt29_2x64d()

### SELECTING AN OPTIMIZER

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001,
                      momentum=0.9, weight_decay=5e-4)

In [ ]:
net = net.to(device)
if device == 'gpu':
    net = torch.nn.DataParallel(net)
    gpunn.benchmark = True

## TRAINING

In [ ]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    running_loss = 0
    running_acc = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)


        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
                # print statistics
        running_loss += loss.item() * inputs.size(0)
    print("Training loss:", running_loss/total)
    print("Training  accuracy is :", 100.*running_acc/total)

In [ ]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
        acc = 100.*correct/total
        print('The test accuracy is :', acc, file=open('log_aug_adam.txt', 'a'))
        print('The test accuracy is :', acc)

        if acc > best_acc:
            print("Saving..")
            state = {
                'net': net.module.state_dict() if isinstance(net, torch.nn.DataParallel) else net.state_dict(),
                'acc': acc,
                'epoch': epoch,
            }
            if not os.path.isdir('checkpoint'):
                os.mkdir('checkpoint')
            torch.save(state, '/content/drive/MyDrive/Resnext_model/checkpoint/ckpts1.pth')
            best_acc = acc


In [1]:
for epoch in range(0,100):
    train(epoch)
    test(epoch)

NameError: name 'train' is not defined